## Notebook for performing active learning to optimize anode-free lithium metal battery electrolytes

## Batch: 1

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, PandasTools, Fragments, rdMolDescriptors, Descriptors
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
# Silence non-critical RDKit warnings to minimize unnecessary outputs
from rdkit import RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel, Matern, ConstantKernel, DotProduct, PairwiseKernel 
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.special import erf
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px

### Reading & standardizing datasets

In [2]:
## in-house dataset (initial labeled dataset for training surrogate models)
df = pd.read_csv('../../datasets/in-house_label_data.csv')
df

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,norm_capacity_14,norm_capacity_15,norm_capacity_16,norm_capacity_17,norm_capacity_18,norm_capacity_19,norm_capacity_20,norm_capacity_21,norm_capacity_22,norm_capacity_23
0,COCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.845301,-0.995151,1.062720,-0.357552,-0.308720,0.309456,0.693325,0.613072,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.183255,-0.948338,0.615257,-0.582269,-1.010187,-0.522075,0.496896,0.301582,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.022233,0.021680,0.022967,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,COCCOCC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.240814,-0.970769,0.671105,-0.607789,-1.124651,-0.436617,0.628824,0.421934,...,0.000573,0.000460,0.000360,0.000407,0.000600,0.000593,0.000447,0.000333,0.000000,0.000000
4,CCOCCOC(C)(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-0.879184,-1.539457,0.507075,-0.111830,0.800175,0.133053,0.926130,-0.208395,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,CCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.092295,-1.519334,-0.056200,-0.093036,-0.572012,-0.583818,-0.053796,-0.111619,...,0.130760,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,CCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.092295,-1.519334,-0.056200,-0.093036,-0.572012,-0.583818,-0.053796,-0.111619,...,0.165080,0.146987,0.135520,0.122253,0.109647,0.104447,0.098607,0.088353,0.000000,0.000000
7,CCCCOCCOC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.067892,-1.253529,0.602357,0.775134,-0.714364,1.295648,0.646532,0.057886,...,0.038067,0.023487,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,COCCOC1CCCC1,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],0.394637,-0.462672,0.854416,-0.416162,-1.007141,1.060702,0.662827,-0.394235,...,0.034467,0.019667,0.019633,0.021167,0.022867,0.026233,0.025507,0.025267,0.000000,0.000000
9,CCCCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],-1.075803,-1.758840,-0.316575,0.667843,-0.912376,0.097061,-0.035434,-0.602637,...,0.051220,0.044827,0.041993,0.037160,0.043613,0.046513,0.048480,0.050413,0.000000,0.000000


In [4]:
X = df.iloc[:,2:27] ## PCA-reduced solvent & salt descriptors and other non-molecular features
y = df['norm_capacity_3'] # normalized discharge capacity at 20th cycle (target variable)
std_scale = StandardScaler().fit(X)
X_std = std_scale.transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

### Active learning workflow

#### Choose best hyperparameters for each kernel

In [5]:
def negative_log_likelihood_rbf(params):
    noise_level, length_scale, alpha = params
    kernel = RBF(length_scale=length_scale)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rq(params):
    noise_level, length_scale, alpha_k, alpha = params 
    kernel = RationalQuadratic(length_scale=length_scale, alpha=alpha_k)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_rbf_expsin(params):
    noise_level, length_scale, periodicity, alpha = params 
    kernel = RBF(length_scale=length_scale) + ExpSineSquared(length_scale=length_scale, periodicity=periodicity)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42) 
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_matern(params):
    noise_level, length_scale, alpha = params
    kernel = Matern(length_scale=length_scale, nu=1.5)
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

def negative_log_likelihood_pairwise(params):
    noise_level, length_scale, alpha = params
    kernel = PairwiseKernel(metric="polynomial")
    white_kernel = WhiteKernel(noise_level=noise_level)
    composite_kernel = kernel + white_kernel
    gpr = GaussianProcessRegressor(kernel=composite_kernel, alpha=alpha, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)
    gpr.fit(X_std, y)
    pred_mean, pred_std = gpr.predict(X_std, return_std=True)
    log_likelihood = np.sum(norm.logpdf(y, loc=pred_mean, scale=pred_std))
    return -log_likelihood

##### Pairwise kernel

In [63]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_pairwise, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.01302141548996467


/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


##### RationalQuadratic kernel

In [67]:
initial_guess = [0.15, 0.01, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha_k, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha_k, alpha
result = minimize(negative_log_likelihood_rq, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha_k, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha_k:", optimized_alpha_k)
print("Optimized alpha:", optimized_alpha)

Optimized noise_level: 0.15000595283478865
Optimized length_scale: 0.010003551490767362
Optimized alpha_k: 0.01
Optimized alpha: 0.01808273981805305


##### Matern-3/2 kernel

In [71]:
initial_guess = [0.15, 0.01, 0.02] # initial guess for noise_level, length_scale, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, alpha
result = minimize(negative_log_likelihood_matern, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/kernels.py:419: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warning

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized alpha: 0.019993933789538003


##### RBF-ExpineSquared kernel

In [75]:
initial_guess = [0.15, 0.01, 1.0, 0.02] # initial guess for noise_level, length_scale, periodicity, alpha
param_bounds = [(1e-4, 1.0), (1e-5, 50.0), (1e-2, 10.0), (1e-4, 0.1)] # bounds for noise_level, length_scale, periodicity, alpha
result = minimize(negative_log_likelihood_rbf_expsin, initial_guess, bounds=param_bounds)
optimized_hyperparameters = result.x
optimized_noise_level, optimized_length_scale, optimized_periodicity, optimized_alpha = optimized_hyperparameters

## enter theses optimized hyperparameters into the combined GP model manually
print("Optimized noise_level:", optimized_noise_level)
print("Optimized length_scale:", optimized_length_scale)
print("Optimized periodicity:", optimized_periodicity)
print("Optimized alpha:", optimized_alpha)

/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of i

Optimized noise_level: 0.15
Optimized length_scale: 0.01
Optimized periodicity: 1.0
Optimized alpha: 0.01878037965954437


#### Train surrogate models

Note: no need to run again, saved model checkpoints have been provided

In [ ]:
## add optimized hyperparameters from previous step
optimized_pairwise_kernel = PairwiseKernel(metric="polynomial") + WhiteKernel(noise_level=0.15)
optimized_matern_kernel = Matern(length_scale=0.01, nu=1.5) + WhiteKernel(noise_level=0.15)
optimized_rbfexpsin_kernel = RBF(length_scale=0.01) + ExpSineSquared(length_scale=0.01, periodicity=1.0) + WhiteKernel(noise_level=0.15)
optimized_rq_kernel = RationalQuadratic(length_scale=0.010003551490767362, alpha=0.01) + WhiteKernel(noise_level=0.15000595283478865)

gpr_models = [GaussianProcessRegressor(kernel=optimized_pairwise_kernel, alpha=0.01302141548996467, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_matern_kernel, alpha=0.019993933789538003, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rq_kernel, alpha=0.01808273981805305, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42),
              GaussianProcessRegressor(kernel=optimized_rbfexpsin_kernel, alpha=0.01878037965954437, optimizer="fmin_l_bfgs_b", n_restarts_optimizer=10, random_state=42)]

model_names = ['../models/batch-1/pairwise_batch1.pkl', '../models/batch-1/matern_batch1.pkl', '../models/batch-1/rq_batch1.pkl', '../models/batch-1/rbf-ess_batch1.pkl']
k = 0
for model in gpr_models:
    print("fitting model: ", k)
    model.fit(X_std, y)
    pickle.dump(model, open(model_names[k], 'wb'))
    k += 1

fitting model:  0


/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/Users/riteshk/codes/miniconda3/envs/mdanalysis/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


fitting model:  1
fitting model:  2
fitting model:  3


#### BMA: aggregate predictions 

##### Acquisition function (Expected improvement)

In [14]:
## final corrected & verified one to be used
def calc_EI(y_pred, y_pred_un, y_pred_un_uncer, epsilon=0.01):
    y_best = np.max(y_pred)
    EI = []
    explore = []
    exploit = []

    for i in range(len(y_pred_un)):
        if y_pred_un_uncer[i] != 0:
            
            # Calculate the cumulative distribution function (CDF) for the Gaussian distribution
            z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            # z = (y_pred_un[i] - y_best - epsilon) / y_pred_un_uncer[i]
            cdf_z = 0.5 * (1 + erf(z / np.sqrt(2)))
            pdf_z = np.exp(-0.5 * z**2) / np.sqrt(2 * np.pi)

            # Calculate Expected Improvement
            expected_improvement = y_pred_un_uncer[i] * (z * cdf_z) + y_pred_un_uncer[i] * pdf_z
            exploitation = y_pred_un_uncer[i] * z * cdf_z
            exploration = y_pred_un_uncer[i] * pdf_z
            EI.append(expected_improvement)
            explore.append(exploration)
            exploit.append(exploitation)
        else:
            EI.append(0.0)
    return EI, exploit, explore

In [29]:
## virtual search space (initial containing 1 million electrolytes)
df_unlabel = pd.read_csv('../datasets/virtual_search_space_1million.csv')
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,salt_ecfp_pca_5,salt_ecfp_pca_6,salt_ecfp_pca_7,salt_ecfp_pca_8,salt_ecfp_pca_9,mol_wt_solv,mol_wt_salt,conc_salt_1,theor_capacity,amt_electrolyte
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,73.052764,186.939685,1.0,150,50
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,114.079313,186.939685,1.0,150,50
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,116.094963,186.939685,1.0,150,50
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,70.058995,186.939685,1.0,150,50
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.128733,-0.322339,0.258767,0.301779,-0.272648,72.044939,186.939685,1.0,150,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,193.077264,144.001775,1.0,150,50
999995,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,240.147392,144.001775,1.0,150,50
999996,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,251.046358,144.001775,1.0,150,50
999997,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,-0.387958,-0.076666,0.008496,0.161338,0.109063,248.133634,144.001775,1.0,150,50


In [30]:
X_un = df_unlabel.iloc[:,2:]
X_un_std = std_scale.transform(X_un)
X_un_std = pd.DataFrame(X_un_std, columns=X_un.columns)

##### Calculate model weights & obtained aggregated mean ($\mu^{aggr}$), uncertainty ($\sigma^{aggr}$), & EI ($EI^{aggr}$)

In [31]:
# Calculate model weights using BMA (first order)
model_names = ['../models/batch-1/pairwise_batch1.pkl', '../models/batch-1/matern_batch1.pkl', '../models/batch-1/rq_batch1.pkl', '../models/batch-1/rbf-ess_batch1.pkl']
model_weights = []
uncertainties = []
predictions = []
y_label_preds = []
for model in model_names:
    gpr = pickle.load(open(model, 'rb'))
    y_un = gpr.predict(X_un_std)
    predictions.append(y_un)
    individual_uncertainties = gpr.predict(X_un_std, return_std=True)[1]
    uncertainties.append(individual_uncertainties)
    likelihoods = norm.pdf(y_un, loc=gpr.predict(X_un_std), scale=individual_uncertainties)
    prior_beliefs = 1.0  # Non-informative prior
    posterior = likelihoods * prior_beliefs
    model_weights.append(posterior / np.sum(posterior))
    y_ = gpr.predict(X_std)
    y_label_preds.append(y_)

In [32]:
df_unlabel['uncertainty_1'] = uncertainties[0]; df_unlabel['uncertainty_2'] = uncertainties[1]; df_unlabel['uncertainty_3'] = uncertainties[2]; df_unlabel['uncertainty_4'] = uncertainties[3]
df_unlabel['prediction_1'] = predictions[0]; df_unlabel['prediction_2'] = predictions[1]; df_unlabel['prediction_3'] = predictions[2]; df_unlabel['prediction_4'] = predictions[3]
df_unlabel['explore_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[2]; df_unlabel['exploit_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[1]
df_unlabel['explore_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[2]; df_unlabel['exploit_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[1]
df_unlabel['explore_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[2]; df_unlabel['exploit_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[1]
df_unlabel['explore_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[2]; df_unlabel['exploit_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[1]
df_unlabel['EI_1'] = calc_EI(y_label_preds[0], predictions[0], uncertainties[0])[0]; df_unlabel['EI_2'] = calc_EI(y_label_preds[1], predictions[1], uncertainties[1])[0]; df_unlabel['EI_3'] = calc_EI(y_label_preds[2], predictions[2], uncertainties[2])[0]; df_unlabel['EI_4'] = calc_EI(y_label_preds[3], predictions[3], uncertainties[3])[0]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,explore_2,exploit_2,explore_3,exploit_3,explore_4,exploit_4,EI_1,EI_2,EI_3,EI_4
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.056091,-0.040843,0.050418,-0.035882,0.036998,-0.029022,0.014181,0.015248,0.014536,0.007976
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.141497,-0.083755,0.054613,-0.039633,0.160430,-0.086233,0.018370,0.057742,0.014981,0.074197
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.039101,-0.031391,0.041169,-0.031303,0.026024,-0.021648,0.010033,0.007710,0.009867,0.004376
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.019404,-0.015850,0.038466,-0.028832,0.010089,-0.008595,0.010635,0.003554,0.009634,0.001494
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.069538,-0.048727,0.053078,-0.037529,0.050186,-0.037945,0.016249,0.020811,0.015549,0.012241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.300623,-0.170906,0.093571,-0.064795,0.389481,-0.181977,0.275982,0.129717,0.028776,0.207505
999995,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.301196,-0.170921,0.094531,-0.065269,0.390023,-0.181731,0.389339,0.130275,0.029262,0.208292
999996,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.300567,-0.170907,0.093543,-0.064791,0.389994,-0.181719,0.281264,0.129660,0.028752,0.208275
999997,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.300908,-0.170914,0.094178,-0.065092,0.389396,-0.182043,0.336648,0.129994,0.029086,0.207353


In [33]:
def calc_aggr_uncer(uncer_1, w_1, pred_1, uncer_2, w_2, pred_2, uncer_3, w_3, pred_3, uncer_4, w_4, pred_4):
    uncer = [uncer_1, uncer_2, uncer_3, uncer_4]
    pred = [pred_1, pred_2, pred_3, pred_4]
    weight = [w_1, w_2, w_3, w_4]
    pred_aggr = w_1 * pred_1 + w_2 * pred_2 + w_3 * pred_3 + w_4 * pred_4
    sum = 0
    for i in range(4):
        sum += weight[i] * (uncer[i]**2 + (pred[i] - pred_aggr)**2)
    aggr_uncer = np.sqrt(sum)
    return aggr_uncer

In [34]:
df_unlabel['prediction_aggr'] = df_unlabel['prediction_1'] * model_weights[0] + df_unlabel['prediction_2'] * model_weights[1] + df_unlabel['prediction_3'] * model_weights[2] + df_unlabel['prediction_4'] * model_weights[3]
df_unlabel['uncertainty_aggr'] = calc_aggr_uncer(df_unlabel['uncertainty_1'], model_weights[0], df_unlabel['prediction_1'], df_unlabel['uncertainty_2'], model_weights[1], df_unlabel['prediction_2'], df_unlabel['uncertainty_3'], model_weights[2], df_unlabel['prediction_3'], df_unlabel['uncertainty_4'], model_weights[3], df_unlabel['prediction_4'])
df_unlabel['explore_aggr'] = df_unlabel['explore_1'] * model_weights[0] + df_unlabel['explore_2'] * model_weights[1] + df_unlabel['explore_3'] * model_weights[2] + df_unlabel['explore_4'] * model_weights[3]
df_unlabel['exploit_aggr'] = df_unlabel['exploit_1'] * model_weights[0] + df_unlabel['exploit_2'] * model_weights[1] + df_unlabel['exploit_3'] * model_weights[2] + df_unlabel['exploit_4'] * model_weights[3]
df_unlabel['ratio_aggr'] = df_unlabel['exploit_aggr'] / df_unlabel['explore_aggr']

## 'EI_aggr' is the final rank by which candidate electrolytes are selected for experimental validation
df_unlabel['EI_aggr'] = df_unlabel['EI_1'] * model_weights[0] + df_unlabel['EI_2'] * model_weights[1] + df_unlabel['EI_3'] * model_weights[2] + df_unlabel['EI_4'] * model_weights[3]
df_unlabel

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
0,CN(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.325380,-0.919052,-0.279367,-0.373535,0.936724,-0.161937,-0.377185,0.259444,...,0.014181,0.015248,0.014536,0.007976,2.952012e-06,0.001313,4.152494e-07,-3.029966e-07,-0.729674,1.122528e-07
1,CN1CCN(C)C1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,1.013958,-0.868963,0.700464,0.728341,0.681048,-1.181697,0.018457,0.793475,...,0.018370,0.057742,0.014981,0.074197,2.190580e-06,0.001335,6.320989e-07,-3.858322e-07,-0.610398,2.462667e-07
2,CN(C)C(=O)N(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.571448,0.079720,-0.117841,-0.342583,1.301206,-0.264839,-0.516489,0.217482,...,0.010033,0.007710,0.009867,0.004376,1.241580e-06,0.001081,2.725457e-07,-2.125616e-07,-0.779912,5.998413e-08
3,CB(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.152976,-1.321495,0.631553,-0.113769,0.728616,-0.486733,-0.311699,0.151570,...,0.010635,0.003554,0.009634,0.001494,3.446851e-06,0.001308,2.643564e-07,-2.034356e-07,-0.769551,6.092076e-08
4,[CH2]N(C)C=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,-0.283297,-0.876057,-0.318283,-0.547668,0.955202,-0.313588,-0.356929,0.299023,...,0.016249,0.020811,0.015549,0.012241,2.888898e-06,0.001327,4.667421e-07,-3.320863e-07,-0.711498,1.346558e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,CC1ON(C)C(C)C1S(C)(=O)=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.771012,-1.078565,0.565077,0.216605,0.613562,-0.742397,-0.257837,0.243970,...,0.275982,0.129717,0.028776,0.207505,-9.982613e-08,0.001937,7.291769e-07,-4.029483e-07,-0.552607,3.262286e-07
999995,COC(=O)C1(N2CCCN(C)C2=O)CCCC1,[Li+].O=C1O[B-](F)(F)OC1=O,0.881747,0.625623,1.808971,0.306581,0.123581,-0.469934,-0.451538,1.337094,...,0.389339,0.130275,0.029262,0.208292,3.151848e-08,0.001881,7.591254e-07,-4.004245e-07,-0.527481,3.587008e-07
999996,COC(=O)N1CCC(OS(C)(=O)=O)CC1=O,[Li+].O=C1O[B-](F)(F)OC1=O,0.700964,0.588889,1.459777,0.806459,-0.042994,-0.081386,0.196396,-0.195416,...,0.281264,0.129660,0.028752,0.208275,-9.545752e-08,0.001939,7.304851e-07,-4.028454e-07,-0.551477,3.276397e-07
999997,CN(C)C(=O)CN(C)C(=O)C1CCCC1(F)F,[Li+].O=C1O[B-](F)(F)OC1=O,0.558235,0.889818,-0.240008,-0.734436,0.749283,0.799906,-0.856418,0.214675,...,0.336648,0.129994,0.029086,0.207353,-2.647618e-08,0.001903,7.464410e-07,-4.030933e-07,-0.540020,3.433477e-07


##### Save top 5000 predictions

In [35]:
df_unlabel_ = df_unlabel.copy()
df_unlabel_ = df_unlabel_.sort_values(by='EI_aggr', ascending=False)
df_unlabel_5000 = df_unlabel_.iloc[:5000,:]
df_unlabel_5000

,solv_comb_sm,salt_comb_sm,solv_ecfp_pca_0,solv_ecfp_pca_1,solv_ecfp_pca_2,solv_ecfp_pca_3,solv_ecfp_pca_4,solv_ecfp_pca_5,solv_ecfp_pca_6,solv_ecfp_pca_7,...,EI_1,EI_2,EI_3,EI_4,prediction_aggr,uncertainty_aggr,explore_aggr,exploit_aggr,ratio_aggr,EI_aggr
302988,COC1(C(F)F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.274757,-1.203490,1.041326,-1.025159,-0.415797,-0.178878,-0.554024,0.520806,...,0.207654,0.312272,0.032396,0.482305,0.000005,0.002300,8.438763e-07,6.042866e-07,0.716084,1.448163e-06
194156,COC1(C(F)(F)F)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.285596,-1.266693,1.241411,-1.102478,0.376095,0.166441,0.024304,0.431072,...,0.205546,0.287685,0.030553,0.455206,0.000005,0.002295,8.434955e-07,5.883782e-07,0.697547,1.431874e-06
297541,COC1(C(C)(F)F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.307584,-1.226729,1.247259,-1.216207,0.634606,0.169550,0.293654,0.228677,...,0.255937,0.284593,0.029139,0.468436,0.000004,0.002259,8.364115e-07,5.906943e-07,0.706225,1.427106e-06
141611,COC1(C(F)F)CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.212383,-1.226273,1.055989,-0.997153,-0.403396,-0.225401,-0.523877,0.563653,...,0.176826,0.300631,0.033008,0.448318,0.000005,0.002308,8.513511e-07,5.730075e-07,0.673057,1.424359e-06
114132,COC1(F)CCCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.431572,-1.193527,1.149880,-0.973532,-0.004855,0.304571,-0.444267,0.611225,...,0.204828,0.308852,0.031455,0.476645,0.000005,0.002275,8.500037e-07,5.640064e-07,0.663534,1.414010e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40064,COC1(N(C)C)CCCN(C=O)C1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.851376,-0.103686,0.435008,-0.624247,0.082175,0.010039,-0.210949,1.345988,...,0.099633,0.160821,0.022038,0.240340,0.000003,0.001721,8.959106e-07,-2.235411e-07,-0.249513,6.723695e-07
281626,CC(C)S(=O)(=O)N1CCCC(=O)C1(C)C,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.919765,-0.738924,0.410033,-0.039115,0.401307,-1.606753,-0.360065,0.420094,...,0.079183,0.155627,0.023258,0.230383,0.000003,0.001751,8.938906e-07,-2.215512e-07,-0.247850,6.723394e-07
271178,COC(C)(C)CN1CCCC1,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.530021,-0.678923,0.777904,-0.079169,0.266731,-0.372377,1.513170,0.303756,...,0.085666,0.112525,0.019859,0.191656,0.000004,0.001807,8.884141e-07,-2.160760e-07,-0.243215,6.723381e-07
55489,CC1(C)CC(F)(F)CCC1=O,[Li+].[N-](S(=O)(=O)F)S(=O)(=O)F,0.535133,-1.218282,0.895559,-0.445502,0.728748,-0.887856,-1.237669,0.645537,...,0.062658,0.146347,0.024031,0.188132,0.000004,0.001818,8.932022e-07,-2.208867e-07,-0.247298,6.723154e-07


In [36]:
df_unlabel_uniq = df_unlabel_5000.drop_duplicates(subset=['solv_comb_sm'], keep='first') ## only keeping unique solvent combinations for selection purposes; these compounds were manually searched in emolecules to finf purchasable compounds
df_unlabel_uniq[['solv_comb_sm', 'salt_comb_sm', 'prediction_aggr', 'uncertainty_aggr', 'explore_aggr', 'exploit_aggr', 'ratio_aggr', 'EI_aggr']].to_csv('../datasets/batch-1/top_5000_suggestions_batch1_uniq_solvents.csv', index=False)